In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

#Настройки показа графиков в блокноте
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)

from sklearn import preprocessing
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

#Стандартизация данных
from sklearn import preprocessing

#Построение диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

from sklearn.model_selection import train_test_split

#Деревья решений для задачи классификации
from sklearn.tree import DecisionTreeClassifier

#Калибровка деревьев решений
from sklearn.calibration import CalibratedClassifierCV

df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')


In [ ]:
df.head().T

In [ ]:
df['Salutation'] = df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
df.Salutation.nunique()
wc = WordCloud(width = 1000,height = 450,background_color = 'white').generate(str(df.Salutation.values))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

df.Salutation.value_counts()

In [ ]:
df.head().T

In [ ]:
#Меняем пропущенные значения на среднее значение по колонке 
grp = df.groupby(['Sex', 'Pclass'])  
df.Age = grp.Age.apply(lambda x: x.fillna(x.median()))

In [ ]:
#Преобразуем объектные столбцы в численные. 
df['Sex'] = df['Sex'].map({'female': 0, 'male': 1})

In [ ]:
df.shape

In [ ]:
df.drop(['Cabin', 'Name', 'Salutation', 'Ticket','Embarked', 'SibSp', 'Parch'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
X_to_be_predicted = df[df.Survived.isnull()]
X_to_be_predicted = X_to_be_predicted.drop(['Survived'], axis = 1)
# X_to_be_predicted[X_to_be_predicted.Age.isnull()]
# X_to_be_predicted.dropna(inplace = True) # 417 x 27
#Training data
train_data = df
train_data = train_data.dropna()
feature_train = train_data['Survived']
label_train = train_data.drop(['Survived'], axis = 1)

In [ ]:
##Random forest
clf = RandomForestClassifier(criterion='entropy',
n_estimators=700,
min_samples_split=10,
min_samples_leaf=1,
max_features='auto',
oob_score=True,
random_state=1,
n_jobs=-1)
x_train, x_test, y_train, y_test = train_test_split(label_train, feature_train, test_size=0.2)
clf.fit(x_train, np.ravel(y_train))
print("RF Accuracy: "+repr(round(clf.score(x_test, y_test) * 100, 2)) + "%")
result_rf=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')
print('The cross validated score for Random forest is:',round(result_rf.mean()*100,2))
y_pred = cross_val_predict(clf,x_train,y_train,cv=10)

In [ ]:
test_data = pd.read_csv("test.csv")
test_data.head()